In [ ]:
!nvidia-smi

In [1]:
!pip install timm

     |████████████████████████████████| 376 kB 4.0 MB/s 


In [2]:
from timm.models.efficientnet import *
import torch

In [31]:
torch.manual_seed(0)
torch.use_deterministic_algorithms(False)
input=torch.randn(1,3,64,64)*255

In [18]:
class Net(torch.nn.Module):
    def __init__(self, verbouse=False):
        super(Net, self).__init__()

        e = efficientnet_b0(pretrained=True, drop_rate=0.2, drop_path_rate=0.2)
        self.verbouse=verbouse
        self.b0 = torch.nn.Sequential(
            e.conv_stem,
            e.bn1,
            e.act1,
        )
        self.b1 = e.blocks[0]
        self.b2 = e.blocks[1]
        self.b3 = e.blocks[2]
        self.b4 = e.blocks[3]
        self.b5 = e.blocks[4]
        self.b6 = e.blocks[5]
        self.b7 = e.blocks[6]
        self.b8 = torch.nn.Sequential(
            e.conv_head,
            e.bn2,
            e.act2,
        )

        self.logit = torch.nn.Linear(1280,1)
       
    # @torch.cuda.amp.autocast()
    def forward(self, image):
        batch_size = len(image)
        x = 2*image-1   
        #if (self.verbouse): print(f'i {x.size()}') 

        x = self.b0(x) 
        #if (self.verbouse): print(f's {x.size()}')
        x = self.b1(x) 
        #if (self.verbouse): print(f'0 {x.size()}')
        x = self.b2(x) 
        #if (self.verbouse): print(f'1 {x.size()}')
        x = self.b3(x)
        #if (self.verbouse): print(f'2 {x.size()}')
        x = self.b4(x) 
        #if (self.verbouse): print(f'3 {x.size()}')
        x = self.b5(x) 
        #if (self.verbouse): print(f'4 {x.size()}')
        x = self.b6(x) 
        #if (self.verbouse): print(f'5 {x.size()}')
        x = self.b7(x) 
        #if (self.verbouse): print(f'6 {x.size()}')
        x = self.b8(x) 
        #if (self.verbouse): print(f'f {x.size()}')
        x = torch.nn.functional.adaptive_avg_pool2d(x,1).reshape(batch_size,-1) 
        #if (self.verbouse): print(f'a {x.size()}')
        x = torch.nn.functional.dropout(x, 0.5, training=self.training)
        #if (self.verbouse): print(f'd {x.size()}')
        logit = self.logit(x)
        return logit

In [35]:
net = Net()

In [36]:
net.eval()
with torch.no_grad():
  logit = net.cuda()(input.cuda())
  print(f'l {logit.size()} {logit}')
  sigm = torch.sigmoid(logit)
  print(f's {sigm.size()} {sigm}')

l torch.Size([1, 1]) tensor([[357.2072]], device='cuda:0')
s torch.Size([1, 1]) tensor([[1.]], device='cuda:0')
